In [ ]:
%pip install numpy==1.24.4
%pip install numba==0.57.1
%pip install shap==0.46.0
%pip install optuna==3.6.1
%pip install scikit-learn==1.3.2
%pip install seaborn==0.13.1
%pip install python-dotenv

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import shap
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

from dotenv import load_dotenv

import pickle

import os


In [ ]:
load_dotenv()

# Accedo a variables de entorno
dataset_path = os.getenv('DATASET_PATH')
dataset_file = os.getenv('DATASET_RAW_FILE')

# Cargo el dataset
data = pd.read_csv(dataset_path + dataset_file)

In [ ]:
# Creamos la clase ternaria
def periodo_anterior(x):
    return x - 1 if x % 100 > 1 else 12 + (x // 100 - 1) * 100

data.sort_values(by=['numero_de_cliente', 'foto_mes'], inplace=True)

# Calcular los periodos
periodo_ultimo = data['foto_mes'].max()
periodo_anteultimo = periodo_anterior(periodo_ultimo)

# Asignar valores comunes a la clase ternaria
data['clase_ternaria'] = data['foto_mes'].apply(lambda x: "CONTINUA" if x < periodo_anteultimo else pd.NA)

# Precomputar tiempo secuencial
data['periodo'] = data['foto_mes'] // 100 * 12 + data['foto_mes'] % 100

last = len(data)

for i in range(last):
    if (data['foto_mes'].iloc[i] <= periodo_anteultimo and i < last - 1 and
        (data['numero_de_cliente'].iloc[i] != data['numero_de_cliente'].iloc[i + 1] or
         (data['numero_de_cliente'].iloc[i] == data['numero_de_cliente'].iloc[i + 1] and
          data['periodo'].iloc[i + 1] > data['periodo'].iloc[i] + 1))):
        data.loc[i, 'clase_ternaria'] = "BAJA+1"

    if (data['foto_mes'].iloc[i] < periodo_anteultimo and i < last - 2 and
        data['numero_de_cliente'].iloc[i] == data['numero_de_cliente'].iloc[i + 1] and
        data['periodo'].iloc[i + 1] == data['periodo'].iloc[i] + 1 and
        (data['numero_de_cliente'].iloc[i + 1] != data['numero_de_cliente'].iloc[i + 2] or
         (data['numero_de_cliente'].iloc[i + 1] == data['numero_de_cliente'].iloc[i + 2] and
          data['periodo'].iloc[i + 2] > data['periodo'].iloc[i + 1] + 2))):
        data.loc[i, 'clase_ternaria'] = "BAJA+2"


In [ ]:
# Imprimo
data['clase_ternaria'].unique()

category_counts = data['clase_ternaria'].value_counts()
category_counts

data.head()

In [ ]:
'''
data['mpayroll_sobre_edad'] = data['mpayroll'] / data['cliente_edad']

# Variables de sumas
data['vm_mfinanciacion_limite'] = data[['Master_mfinanciacion_limite', 'Visa_mfinanciacion_limite']].sum(axis=1, skipna=True)
data['vm_Fvencimiento'] = data[['Master_Fvencimiento', 'Visa_Fvencimiento']].min(axis=1, skipna=True)
data['vm_Finiciomora'] = data[['Master_Finiciomora', 'Visa_Finiciomora']].min(axis=1, skipna=True)
data['vm_msaldototal'] = data[['Master_msaldototal', 'Visa_msaldototal']].sum(axis=1, skipna=True)
data['vm_msaldopesos'] = data[['Master_msaldopesos', 'Visa_msaldopesos']].sum(axis=1, skipna=True)
data['vm_msaldodolares'] = data[['Master_msaldodolares', 'Visa_msaldodolares']].sum(axis=1, skipna=True)
data['vm_mconsumospesos'] = data[['Master_mconsumospesos', 'Visa_mconsumospesos']].sum(axis=1, skipna=True)
data['vm_mconsumosdolares'] = data[['Master_mconsumosdolares', 'Visa_mconsumosdolares']].sum(axis=1, skipna=True)
data['vm_mlimitecompra'] = data[['Master_mlimitecompra', 'Visa_mlimitecompra']].sum(axis=1, skipna=True)
data['vm_madelantopesos'] = data[['Master_madelantopesos', 'Visa_madelantopesos']].sum(axis=1, skipna=True)
data['vm_madelantodolares'] = data[['Master_madelantodolares', 'Visa_madelantodolares']].sum(axis=1, skipna=True)
data['vm_fultimo_cierre'] = data[['Master_fultimo_cierre', 'Visa_fultimo_cierre']].max(axis=1, skipna=True)
data['vm_mpagado'] = data[['Master_mpagado', 'Visa_mpagado']].sum(axis=1, skipna=True)
data['vm_mpagospesos'] = data[['Master_mpagospesos', 'Visa_mpagospesos']].sum(axis=1, skipna=True)
data['vm_mpagosdolares'] = data[['Master_mpagosdolares', 'Visa_mpagosdolares']].sum(axis=1, skipna=True)
data['vm_fechaalta'] = data[['Master_fechaalta', 'Visa_fechaalta']].max(axis=1, skipna=True)
data['vm_mconsumototal'] = data[['Master_mconsumototal', 'Visa_mconsumototal']].sum(axis=1, skipna=True)
data['vm_cconsumos'] = data[['Master_cconsumos', 'Visa_cconsumos']].sum(axis=1, skipna=True)
data['vm_cadelantosefectivo'] = data[['Master_cadelantosefectivo', 'Visa_cadelantosefectivo']].sum(axis=1, skipna=True)
data['vm_mpagominimo'] = data[['Master_mpagominimo', 'Visa_mpagominimo']].sum(axis=1, skipna=True)

# Variables de ratios
data['vmr_Master_mlimitecompra'] = data['Master_mlimitecompra'] / data['vm_mlimitecompra']
data['vmr_Visa_mlimitecompra'] = data['Visa_mlimitecompra'] / data['vm_mlimitecompra']
data['vmr_msaldototal'] = data['vm_msaldototal'] / data['vm_mlimitecompra']
data['vmr_msaldopesos'] = data['vm_msaldopesos'] / data['vm_mlimitecompra']
data['vmr_msaldopesos2'] = data['vm_msaldopesos'] / data['vm_msaldototal']
data['vmr_msaldodolares'] = data['vm_msaldodolares'] / data['vm_mlimitecompra']
data['vmr_msaldodolares2'] = data['vm_msaldodolares'] / data['vm_msaldototal']
data['vmr_mconsumospesos'] = data['vm_mconsumospesos'] / data['vm_mlimitecompra']
data['vmr_mconsumosdolares'] = data['vm_mconsumosdolares'] / data['vm_mlimitecompra']
data['vmr_madelantopesos'] = data['vm_madelantopesos'] / data['vm_mlimitecompra']
data['vmr_madelantodolares'] = data['vm_madelantodolares'] / data['vm_mlimitecompra']
data['vmr_mpagado'] = data['vm_mpagado'] / data['vm_mlimitecompra']
data['vmr_mpagospesos'] = data['vm_mpagospesos'] / data['vm_mlimitecompra']
data['vmr_mpagosdolares'] = data['vm_mpagosdolares'] / data['vm_mlimitecompra']
data['vmr_mconsumototal'] = data['vm_mconsumototal'] / data['vm_mlimitecompra']
data['vmr_mpagominimo'] = data['vm_mpagominimo'] / data['vm_mlimitecompra']



# Filtramos solo las columnas numéricas
numeric_cols = data.select_dtypes(include=[np.number])

# Reemplazo valores infinitos con NaN solo en las columnas numéricas
infinitos_qty = np.isinf(numeric_cols).sum().sum()
if infinitos_qty > 0:
    print(f"ATENCIÓN: Hay {infinitos_qty} valores infinitos en tu dataset. Serán pasados a NaN.")
    data[numeric_cols.columns] = numeric_cols.replace([np.inf, -np.inf], np.nan)

# Valvula de seguridad para evitar valores NaN
nans_qty = data.isna().sum().sum()
if nans_qty > 0:
    print(f"ATENCIÓN: Hay {nans_qty} valores NaN en tu dataset. Serán pasados arbitrariamente a 0.")
    # Reemplazo valores NaN con 0
    data.fillna(0, inplace=True)

data.head()
'''

In [ ]:
'''
# Definimos las columnas que NO vamos a utilizar para crear los lags
campitos = ["numero_de_cliente", "foto_mes", "clase_ternaria"]

# Seleccionamos todas las columnas lagueables (que no están en campitos)
cols_lagueables = [col for col in data.columns if col not in campitos]  

# Ordenamos el DataFrame por 'numero_de_cliente' y 'foto_mes'
data.sort_values(by=["numero_de_cliente", "foto_mes"], inplace=True)

# Creamos los lags de orden 1 para las columnas en cols_lagueables
#for col in cols_lagueables:
    # Creamos la columna lag1 (desplazamiento hacia abajo de 1 periodo)
#    data[f"{col}_lag1"] = data.groupby("numero_de_cliente")[col].shift(1)

# Creamos los delta lags de orden 1
#for col in cols_lagueables:
    # Calculamos la diferencia entre el valor actual y el valor lag1
#    data[f"{col}_delta1"] = data[col] - data[f"{col}_lag1"]

# Creamos un array vacío para los nuevos datos (lags y deltas)
lags = {}
deltas = {}

# Creamos los lags y deltas usando NumPy para evitar la fragmentación
for col in cols_lagueables:
    # Obtenemos el grupo por 'numero_de_cliente'
    grouped = data.groupby("numero_de_cliente")[col]
    
    # Calculamos el lag usando shift con NumPy
    lags[f"{col}_lag1"] = grouped.shift(1).to_numpy()
    
    # Calculamos el delta como la diferencia actual - lag1
    deltas[f"{col}_delta1"] = data[col].to_numpy() - lags[f"{col}_lag1"]

# Asignamos los resultados de lag y delta directamente al DataFrame
for col_lag, values_lag in lags.items():
    data[col_lag] = values_lag
    
for col_delta, values_delta in deltas.items():
    data[col_delta] = values_delta


data.head()
'''

In [ ]:
# Hacemos un modelo LGBM para predecir la importancia de las columnas

Xtrain = data[data['foto_mes'].isin([202104, 202105, 202106])]
ytrain = Xtrain["clase_ternaria"].map(lambda x: 0 if x == "CONTINUA" else 1)

Xtrain = Xtrain.drop("clase_ternaria", axis=1)

lgb_train = lgb.Dataset(Xtrain, ytrain)

params = {
    'objective': 'binary',
    'learning_rate': 0.01,
    'verbose': 2,
    'max_bin': 15,
    'min_data_in_leaf': 500,
    'verbose': 0,
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100)

lgbm_importancia = pd.DataFrame({'Features': gbm.feature_name(),
                        'Importances': gbm.feature_importance()})
lgbm_importancia.sort_values(by='Importances', inplace=True, ascending=False)
lgbm_importancia

In [ ]:
# Calculo la importancia de las variables usando SHAP

explainer = shap.TreeExplainer(gbm)
shap_values = explainer.shap_values(Xtrain)

shap_df = pd.DataFrame(shap_values, columns = Xtrain.columns)
shap_df

shap_importancia = pd.DataFrame(np.abs(shap_values).mean(0), columns=['SHAP Importance'])
shap_importancia['Feature'] = Xtrain.columns
shap_importancia.sort_values('SHAP Importance', ascending=False, inplace=True)
shap_importancia

In [ ]:
# Visualizo

shap.summary_plot(shap_values, Xtrain)

In [ ]:
'''
data.drop([
    'cprestamos_personales',
    'mprestamos_personales',
    'Master_msaldodolares',
    'Master_mconsumototal',
    'Visa_Finiciomora',
    'Visa_delinquency',
    'Visa_msaldodolares',
    'Master_cadelantosefectivo',
    'Visa_cadelantosefectivo',
    'matm',
    'catm_trx_other',
    'matm_other',
    'catm_trx',
    'Master_cconsumos',
    'cmobile_app_trx',
    'Visa_mpagosdolares',
    'Master_delinquency',
    'Visa_mconsumototal',
    'Visa_madelantopesos',
    'Master_mpagado',
    'Master_madelantodolares',
    'Master_madelantopesos',
    'Visa_madelantodolares',
    'Master_Finiciomora',
    'Visa_fultimo_cierre',
    'ccajas_otras',
    'mforex_buy',
    'ccajas_extracciones',
    'ctarjeta_master_debitos_automaticos',
    'cpayroll2_trx',
    'mpayroll2',
    'cseguro_accidentes_personales',
    'cseguro_vivienda',
    'cseguro_auto',
    'cseguro_vida',
    'minversion2',
    'cinversion2',
    'minversion1_dolares',
    'minversion1_pesos',
    'cinversion1',
    'mplazo_fijo_pesos',
    'mplazo_fijo_dolares',
    'cplazo_fijo',
    'mprestamos_hipotecarios',
    'cprestamos_hipotecarios',
    'mprestamos_prendarios',
    'cprestamos_prendarios',
    'ctarjeta_debito_transacciones',
    'mcaja_ahorro_adicional',
    'mcuenta_corriente_adicional',
    'ccuenta_corriente',
    'cliente_vip',
    'mttarjeta_visa_debitos_automaticos',
    'mttarjeta_master_debitos_automaticos',
    'ccajas_depositos',
    'cpagodeservicios',
    'mcheques_emitidos_rechazados',
    'ccheques_emitidos_rechazados',
    'mcheques_depositados_rechazados',
    'ccheques_depositados_rechazados',
    'mcheques_emitidos',
    'ccheques_emitidos',
    'mcheques_depositados',
    'ccheques_depositados',
    'mextraccion_autoservicio',
    'mtransferencias_emitidas',
    'mforex_sell',
    'cforex_sell',
    'cforex_buy',
    'cforex',
    'mtarjeta_master_descuentos',
    'ctarjeta_master_descuentos',
    'mtarjeta_visa_descuentos',
    'ctarjeta_visa_descuentos',
    'mcajeros_propios_descuentos',
    'ccajeros_propios_descuentos',
    'cpagomiscuentas',
    'mpagodeservicios',
    'Master_mfinanciacion_limite'
], axis=1, inplace=True)
'''

data.drop([
    'cprestamos_personales',
    'mprestamos_personales',
], axis=1, inplace=True)

data['clase_ternaria'].fillna('BAJA+2', inplace=True)

In [ ]:
#Imprime primeras 5 filas del df

data.head()

In [ ]:
# Especificar la ruta completa del archivo donde deseas guardar el DataFrame
output_file = dataset_path + "competencia_02.csv"

# Guardar el DataFrame como un archivo CSV en la ruta especificada
data.to_csv(output_file, index=False)